# Movie Lens Recommender Intuition Purpose

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [20]:
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [23]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [24]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [25]:
data.describe().show()

data.filter('userid > 28').show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      3|   1.0|    29|
|      4|   1.0|    29|
|      5|   1.0|    29|
|      7|   2.0|    29|
|      9|   1.0|    29|
|     10|   3.0|    29|
|     11|   1.0|    29|
|     13|   3.0|    29|
|     14|   1.0|    29|
|     15|   1.0|    29|
|     17|   3.0|    29|
|     19|   3.0|    29|
|     22|   3.0|    29|
|     23|   

We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [26]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [34]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

test.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|   1.0|    15|
|      0|   1.0|    27|
|      0|   3.0|    10|
|      1|   1.0|     3|
|      1|   1.0|    14|
|      1|   1.0|    18|
|      1|   1.0|    19|
|      1|   1.0|    26|
|      2|   1.0|    25|
|      2|   4.0|    21|
|      3|   1.0|     0|
|      3|   3.0|    14|
|      4|   1.0|     9|
|      4|   1.0|    23|
|      4|   2.0|     1|
|      6|   1.0|     6|
|      6|   3.0|    26|
|      7|   1.0|     7|
|      7|   1.0|    14|
|      7|   1.0|    15|
+-------+------+------+
only showing top 20 rows



Now let's see hwo the model performed!

In [28]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [29]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26| 1.0907565|
|     31|   1.0|    27|0.12743771|
|     31|   1.0|    13| 1.1275916|
|     31|   1.0|     4|  2.299378|
|     31|   1.0|    18| 0.5967964|
|     85|   2.0|    20| 3.3108144|
|     78|   1.0|    28| 1.5271351|
|     78|   1.0|     1| 1.0260153|
|     78|   1.0|    20| 1.0570037|
|     78|   1.0|    24| 1.2871966|
|     34|   3.0|    25| 0.5749709|
|     34|   1.0|    14|  3.846885|
|     81|   1.0|     3| 0.9909062|
|     81|   2.0|    29|-1.4014562|
|     81|   1.0|    21| 3.4766488|
|     76|   1.0|    26|-1.7491782|
|     76|   3.0|     3| 2.6228201|
|     76|   1.0|    11| 3.8375545|
|     26|   1.0|    13| 2.2489018|
|     26|   1.0|     9|  3.324199|
+-------+------+------+----------+
only showing top 20 rows



In [30]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.2470419620450794


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [31]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [32]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     20|    11|
|     23|    11|
|     41|    11|
|     45|    11|
|     61|    11|
|     69|    11|
|     71|    11|
|     76|    11|
|     77|    11|
|     80|    11|
+-------+------+



In [33]:
reccomendations = model.transform(single_user)

In [54]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     30|    11|  5.578189|
|     13|    11|  3.257565|
|     70|    11| 2.7580981|
|     99|    11| 1.7420897|
|     18|    11| 1.5150304|
|     75|    11|   1.34218|
|     79|    11| 0.9733073|
|     66|    11| 0.5732717|
|     78|    11| 0.4434041|
|      0|    11|  -1.85298|
+-------+------+----------+

